# PA3b Sentiment Classification
#### Applied Machine Learning
Grpup 39: Sebastian Kölbel & Min Ze Teh

We begin by importing libraries needed for the task

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# for converting training and test datasets into matrices
from sklearn.feature_extraction.text import TfidfVectorizer

## Task1
We begin by importing the testing and training csv files using pandas and dividing the sets into input and output.

In [71]:
cs_training = pd.read_csv('Data/crowdsourced_train.csv')
X_cs_train = cs_training.drop('sentiment', axis=1)
Y_cs_train = cs_training['sentiment']

gold_training = pd.read_csv('Data/gold_train.csv')
X_gold_train = gold_training.drop('sentiment', axis=1)
Y_gold_train = gold_training['sentiment']


testing = pd.read_csv('Data/test.csv')
X_test = testing.drop('sentiment', axis=1)
Y_test = testing['sentiment']

We probably want to pre-process these a bit. make sure the sentiments is only 3 different values and make sure that the input text is perhaps all lowercase letters?
Separate punctuation from word?


In [ ]:
def train_document_classifier(X, Y):
    pipeline = make_pipeline( TfidfVectorizer(), LinearSVC(dual='auto') )
    pipeline.fit(X, Y)
    return pipeline